<a href="https://colab.research.google.com/github/baller01/Project-4/blob/main/CO2_emissions_optimized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow

In [2]:
# Import the modules
import pandas as pd
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [3]:
# Read the csv file into a pandas DataFrame
carbon_df = pd.read_csv("CO2_Emissions_Canada.csv")

# Review the DataFrame
carbon_df.head()

,Make,Model,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),CO2 Emissions(g/km)
0,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
2,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136
3,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255
4,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244


In [4]:
carbon_df.dtypes

Make                                 object
Model                                object
Vehicle Class                        object
Engine Size(L)                      float64
Cylinders                             int64
Transmission                         object
Fuel Type                            object
Fuel Consumption City (L/100 km)    float64
Fuel Consumption Hwy (L/100 km)     float64
Fuel Consumption Comb (L/100 km)    float64
Fuel Consumption Comb (mpg)           int64
CO2 Emissions(g/km)                   int64
dtype: object

In [5]:
co2_df=carbon_df.drop(columns=['Make', 'Model', 'Fuel Consumption Comb (L/100 km)', 'Fuel Consumption Comb (mpg)'])
co2_df

,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),CO2 Emissions(g/km)
0,COMPACT,2.0,4,AS5,Z,9.9,6.7,196
1,COMPACT,2.4,4,M6,Z,11.2,7.7,221
2,COMPACT,1.5,4,AV7,Z,6.0,5.8,136
3,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,255
4,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,244
...,...,...,...,...,...,...,...,...
7380,SUV - SMALL,2.0,4,AS8,Z,10.7,7.7,219
7381,SUV - SMALL,2.0,4,AS8,Z,11.2,8.3,232
7382,SUV - SMALL,2.0,4,AS8,Z,11.7,8.6,240
7383,SUV - STANDARD,2.0,4,AS8,Z,11.2,8.3,232


In [ ]:
#Evaluating composition of the column by value counts
co2_df["Vehicle Class"].value_counts()

SUV - SMALL                 1217
MID-SIZE                    1133
COMPACT                     1022
SUV - STANDARD               735
FULL-SIZE                    639
SUBCOMPACT                   606
PICKUP TRUCK - STANDARD      538
TWO-SEATER                   460
MINICOMPACT                  326
STATION WAGON - SMALL        252
PICKUP TRUCK - SMALL         159
MINIVAN                       80
SPECIAL PURPOSE VEHICLE       77
VAN - PASSENGER               66
STATION WAGON - MID-SIZE      53
VAN - CARGO                   22
Name: Vehicle Class, dtype: int64

In [ ]:
#Evaluating composition of the column by value counts
co2_df["Transmission"].value_counts()

AS6     1324
AS8     1211
M6       901
A6       789
A8       490
AM7      445
A9       339
AS7      319
AV       295
M5       193
AS10     168
AM6      132
AV7      118
AV6      113
M7        91
A5        84
AS9       77
A4        65
AM8       62
A7        53
AV8       39
A10       31
AS5       26
AV10      11
AM5        4
AM9        3
AS4        2
Name: Transmission, dtype: int64

In [ ]:
#Binning of the column "Transmission" for values below 20
# use the variable name `classifications_to_replace`
Transmission_to_replace =co2_df["Transmission"].value_counts().loc[lambda x: x<20].index

# Replace in dataframe
for t in Transmission_to_replace:
    co2_df['Transmission'] = co2_df['Transmission'].replace(t,"Other")

# Check to make sure binning was successful
co2_df['Transmission'].value_counts()

AS6      1324
AS8      1211
M6        901
A6        789
A8        490
AM7       445
A9        339
AS7       319
AV        295
M5        193
AS10      168
AM6       132
AV7       118
AV6       113
M7         91
A5         84
AS9        77
A4         65
AM8        62
A7         53
AV8        39
A10        31
AS5        26
Other      20
Name: Transmission, dtype: int64

In [ ]:
#Evaluating composition of the column by value counts
co2_df["Engine Size(L)"].value_counts()

2.0    1460
3.0     804
3.6     536
3.5     529
2.5     423
2.4     346
1.6     302
5.3     290
1.8     216
1.4     211
5.0     207
1.5     207
4.0     198
6.2     162
4.4     131
5.7     116
3.8     114
3.7     112
2.7      99
6.0      94
3.3      85
4.7      78
3.4      52
5.5      52
4.3      52
2.3      51
5.2      48
6.4      46
5.6      36
3.2      35
4.8      32
4.6      32
6.6      29
1.2      25
6.7      25
2.8      24
2.9      23
1.0      18
6.5      18
4.2      14
1.3      11
5.4       8
6.8       8
2.1       5
8.4       5
5.9       5
6.3       3
0.9       3
8.0       3
5.8       1
2.2       1
Name: Engine Size(L), dtype: int64

In [ ]:
co2_df.loc[co2_df["CO2 Emissions(g/km)"] <= 255, "CO2"] = 1
co2_df.loc[co2_df["CO2 Emissions(g/km)"] > 255, "CO2"] = 0
co2_df
print(co2_df["CO2"].value_counts())

1.0    4182
0.0    3203
Name: CO2, dtype: int64


In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
# Scaling the numeric columns
co2_scaled_df= StandardScaler().fit_transform(co2_df[["Engine Size(L)", "Cylinders", "Fuel Consumption City (L/100 km)", "Fuel Consumption Hwy (L/100 km)"]])
# Creating a DataFrame with with the scaled data
co2_transformed_df= pd.DataFrame(co2_scaled_df, columns= ["Engine Size(L)", "Cylinders", "Fuel Consumption City (L/100 km)", "Fuel Consumption Hwy (L/100 km)"])

co2_transformed_df

,Engine Size(L),Cylinders,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km)
0,-0.856721,-0.883408,-0.759002,-1.052781
1,-0.561317,-0.883408,-0.387577,-0.603202
2,-1.225976,-0.883408,-1.873275,-1.457401
3,0.251043,0.210575,0.040990,0.026208
4,0.251043,0.210575,-0.130437,-0.153624
...,...,...,...,...
7380,-0.856721,-0.883408,-0.530433,-0.603202
7381,-0.856721,-0.883408,-0.387577,-0.333455
7382,-0.856721,-0.883408,-0.244721,-0.198582
7383,-0.856721,-0.883408,-0.387577,-0.333455


In [ ]:
# Transform the Card Type column using get_dummies()
dummies_df=pd.get_dummies(co2_df[["Vehicle Class", "Transmission","Fuel Type"]])
dummies_df.head()

,Vehicle Class_COMPACT,Vehicle Class_FULL-SIZE,Vehicle Class_MID-SIZE,Vehicle Class_MINICOMPACT,Vehicle Class_MINIVAN,Vehicle Class_PICKUP TRUCK - SMALL,Vehicle Class_PICKUP TRUCK - STANDARD,Vehicle Class_SPECIAL PURPOSE VEHICLE,Vehicle Class_STATION WAGON - MID-SIZE,Vehicle Class_STATION WAGON - SMALL,...,Transmission_AV8,Transmission_M5,Transmission_M6,Transmission_M7,Transmission_Other,Fuel Type_D,Fuel Type_E,Fuel Type_N,Fuel Type_X,Fuel Type_Z
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
# Concatenate the df_shopping_transformed and the card_dummies DataFrames
co2_new_df=pd.concat([co2_transformed_df, dummies_df], axis=1)
co2_new_df

,Engine Size(L),Cylinders,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Vehicle Class_COMPACT,Vehicle Class_FULL-SIZE,Vehicle Class_MID-SIZE,Vehicle Class_MINICOMPACT,Vehicle Class_MINIVAN,Vehicle Class_PICKUP TRUCK - SMALL,...,Transmission_AV8,Transmission_M5,Transmission_M6,Transmission_M7,Transmission_Other,Fuel Type_D,Fuel Type_E,Fuel Type_N,Fuel Type_X,Fuel Type_Z
0,-0.856721,-0.883408,-0.759002,-1.052781,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,-0.561317,-0.883408,-0.387577,-0.603202,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,-1.225976,-0.883408,-1.873275,-1.457401,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0.251043,0.210575,0.040990,0.026208,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0.251043,0.210575,-0.130437,-0.153624,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7380,-0.856721,-0.883408,-0.530433,-0.603202,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7381,-0.856721,-0.883408,-0.387577,-0.333455,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7382,-0.856721,-0.883408,-0.244721,-0.198582,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7383,-0.856721,-0.883408,-0.387577,-0.333455,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
# Split our preprocessed data into our features and target arrays
X=co2_new_df.values
y=co2_df["CO2"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test= train_test_split(X,y, random_state= 1)

In [ ]:
##Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=20, activation="relu", input_dim= input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=20, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 20)                1000      
                                                                 
 dense_13 (Dense)            (None, 20)                420       
                                                                 
 dense_14 (Dense)            (None, 1)                 21        
                                                                 
Total params: 1,441
Trainable params: 1,441
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=50)

Epoch 1/50
174/174 [==============================] - 2s 3ms/step - loss: 0.3223 - accuracy: 0.8953
Epoch 2/50
174/174 [==============================] - 1s 3ms/step - loss: 0.1484 - accuracy: 0.9426
Epoch 3/50
174/174 [==============================] - 0s 3ms/step - loss: 0.1149 - accuracy: 0.9549
Epoch 4/50
174/174 [==============================] - 0s 3ms/step - loss: 0.0873 - accuracy: 0.9644
Epoch 5/50
174/174 [==============================] - 0s 2ms/step - loss: 0.0664 - accuracy: 0.9762
Epoch 6/50
174/174 [==============================] - 0s 2ms/step - loss: 0.0545 - accuracy: 0.9787
Epoch 7/50
174/174 [==============================] - 0s 2ms/step - loss: 0.0483 - accuracy: 0.9801
Epoch 8/50
174/174 [==============================] - 0s 2ms/step - loss: 0.0438 - accuracy: 0.9816
Epoch 9/50
174/174 [==============================] - 0s 2ms/step - loss: 0.0400 - accuracy: 0.9834
Epoch 10/50
174/174 [==============================] - 0s 2ms/step - loss: 0.0387 - accuracy: 0.9839

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test, y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

58/58 - 0s - loss: 0.0461 - accuracy: 0.9821 - 218ms/epoch - 4ms/step
Loss: 0.046138204634189606, Accuracy: 0.9821332097053528


### The first model had 1 layer and without binning, had a Loss of 0.043466 an accuracy 0.981591 and . Optimization was attempted by adding layers to 2, and binning the "Transmisssion" column.